## 0. Setup of the environment

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

scripts_path = os.path.abspath(os.path.join('helpers'))
if scripts_path not in sys.path:
    sys.path.insert(0,scripts_path)
from station_location import *

In [2]:
DATA_PATH = 'data/'
PROCESSED_DATA_PATH = 'processed-data'
TRUE_VALUES_PATHS = ['data/catalog_2017.csv', 
                     'data/catalog_2018.csv', 
                     'data/catalog_2019.csv']
ANTENNA_INDEX = 0
SPS = 100
START_TIME_INDEX = 3
TIME_WINDOW = 60
NUM_DAY_PER_DIR = 15

# The antenna we chose, as in the scripts
ANTENNA = ['GS', 'OK029', '00', 'HH1', 'M']

In [3]:
def get_timestamps(start_time, number_window):
    """
    Function useful to get a list of timestamps from a given start time.
    """
    times = [start_time + pd.to_timedelta(i * (total_window_size / SPS), unit='s') 
             for i in range(number_window)]
    return pd.Series(times)

## 1. Creation of the catalog of earthquakes

First we load the raw hand-made catalog

In [4]:
catalog = pd.DataFrame()
for ys in TRUE_VALUES_PATHS:
    new_catalog = pd.read_csv(ys)
    if 'depth' in new_catalog.columns:
        new_catalog.rename(columns={'depth': 'depth_km'}, inplace=True)
    catalog = catalog.append(new_catalog, sort=False) 
catalog = catalog.reset_index()
catalog["origintime"] = pd.to_datetime(catalog["origintime"])
catalog.sort_values("origintime", inplace=True)
catalog.head(5)

index       id                 origintime  latitude  longitude  depth_km  \
0      0  27287.0 2017-01-01 02:29:41.795999  35.25422  -97.76945     4.194   
1      1  27285.0 2017-01-01 06:43:01.465000  36.28558  -97.50169     4.179   
2      2  27278.0 2017-01-02 15:25:35.006000  36.39678  -96.88587     4.152   
3      3  27277.0 2017-01-02 15:28:40.168999  36.39613  -96.88583     4.241   
4      4  27001.0 2017-01-02 15:41:50.164000  36.30314  -97.53593     3.112   

   err_lon  err_lat  err_depth err_origintime  ...  \
0      0.5      0.4        0.9           0.31  ...   
1      0.4      0.3        0.9           0.37  ...   
2      0.3      0.2        0.4           0.32  ...   
3      0.4      0.3        0.6            0.4  ...   
4      0.3      0.3        0.9           0.36  ...   

                                             reafile  \
0  /home/analyst/REA/OGS__/2017/01/01-0228-56L.S2...   
1  /home/analyst/REA/OGS__/2017/01/01-0642-16L.S2...   
2  /home/analyst/REA/OGS__/2017/01/02-1524-49L.S2...   
3  /home/analyst/REA/OGS__/2017/01/02-1527-55L.S2...   
4  /home/analyst/REA/OGS__/2017/01/02-1541-15L.S2...   

                     reamtime  \
0   2017-01-27 21:31:51.28688   
1  2017-01-27 21:05:04.611471   
2  2017-01-27 20:11:43.272473   
3  2017-01-27 19:22:46.231692   
4  2017-01-12 17:05:31.794816   

                                                geom    pdlid mw_ogs  \
0  0101000020E6100000107A36AB3E7158C09BFEEC478AA0...  18702.0    NaN   
1  0101000020E6100000BCAE5FB01B6058C02332ACE28D24...  18704.0    NaN   
2  0101000020E61000007BA01518B23858C07BDAE1AFC932...  18691.0    NaN   
3  0101000020E610000034F44F70B13858C0747B4963B432...  18690.0    NaN   
4  0101000020E61000007FBC57AD4C6258C0020EA14ACD26...  18560.0    NaN   

   event_id magnitude  magnitude_source  state  status  
0       NaN       NaN               NaN    NaN     NaN  
1       NaN       NaN               NaN    NaN     NaN  
2       NaN       NaN               NaN    NaN     NaN  
3       NaN       NaN               NaN    NaN     NaN  
4       NaN       NaN               NaN    NaN     NaN  

[5 rows x 36 columns]

There's a lot of attribute in this catalog, however only the first four columns will be useful for us. The origin time of the earthquake of course as we will create the $y$ values based on it. The other ones will be useful to calibrate the catalog to our station.

We take care of that now, by first defining the station and then delaying the catalog accordingly

*Note: as the graph is interactive, you may have to rerun the cell below to be able to see it.*

In [37]:
import folium

network = ANTENNA[0]
station = ANTENNA[1]
channel = ANTENNA[3]
lat_station, lon_station, alt_station = get_location(network, station, channel)
print(f"Location our station in Oklahoma:")
print(f"\tlatitude: {lat_station}\n\tlongitude: {lon_station}\n\televation: {alt_station}")
m = folium.Map(
    location=[lat_station, lon_station],
    zoom_start=6,
    tiles='Stamen Terrain'
)
folium.Marker(
    location=[lat_station, lon_station],
    popup='Station'
).add_to(m)
m

Location our station in Oklahoma:
	latitude: 35.79657
	longitude: -97.454857
	elevation: 333.0


Now in order to delay the catalog, we must first compute the speed of of propagation of the waves. We do that empirically and manually. We picked three earthquake and manually reported the delay between the earthquake in the ObsPy dataset for our station and the catalog. Those earthquake have been chosen so as to one of them is nearthe station, one far away and one between the two. We also add the distances to the earthquakes to the catalog. This allows us to find the speed of the propagation based on those three earthquakes. Then using this speed average, the entire catalog can be calibrated.

In [9]:
def add_distances_to_catalog(catalog, lat_station, lon_station, alt_station):
    catalog["distance"] = catalog.loc[:, ["latitude", "longitude", "depth_km"]].apply(
        lambda row: compute_distance(lat_station, lon_station, alt_station, row[0], row[1], -1000*row[2]), axis=1)

In [10]:
add_distances_to_catalog(catalog, lat_station, lon_station, alt_station)

print('Speed of propagation:')

eq_speed_1_id = 3336
eq_speed_1 = catalog[catalog.index == eq_speed_1_id]
dist_1 = float(eq_speed_1['distance'])
time_1 = 9.039            # The manually extracted delay
speed_1 = dist_1/time_1
print('\tEarthquake 1:', speed_1, 'm/s')

eq_speed_2_id = 3362
eq_speed_2 = catalog[catalog.index == eq_speed_2_id]
dist_2 = float(eq_speed_2['distance'])
time_2 = 20.283
speed_2 = dist_2/time_2
print('\tEarthquake 2:', speed_2, 'm/s')

eq_speed_3_id = 3356
eq_speed_3 = catalog[catalog.index == eq_speed_3_id]
dist_3 = float(eq_speed_3['distance'])
time_3 = 5.78
speed_3 = dist_3/time_3
print('\tEarthquake 3:', speed_3, 'm/s')

speed_average = (speed_1 + speed_2 + speed_3)/3
print('Average speed: ', speed_average, 'm/s')

Speed of propagation:
	Earthquake 1: 5335.284683852213 m/s
	Earthquake 2: 5814.270050347443 m/s
	Earthquake 3: 5742.549681025605 m/s
Average speed:  5630.701471741754 m/s


We obtain approximately the same value than reported in paper for p-waves, that waves we are dealing with, thus we are confident with this value even tho it is based on only three samples.

In [11]:
def change_time_according_to_station(catalog, lat_station, lon_station, alt_station, speed_average):
    catalog["time_diff"] = catalog["distance"].map(lambda dist: dist/speed_average)
    catalog["origintime"] = catalog.loc[:, ["origintime", "time_diff"]].apply(lambda row: row[0] + pd.to_timedelta(row[1], unit='s'), axis=1)
    
change_time_according_to_station(catalog, lat_station, lon_station, alt_station, speed_average)

We have now a catalog that is adjusted to our station and thus to our data. We now compute the features from the raw data.

## 2. Creation of the features

The architecture for loading the data is a bit complicated, but it is needed as it cannot be done in one pass because that it does not fit in memory. Also, one additionnal difficulty is that we have to expand the raw data to fill in the gaps by interpolating the wave between every file (corresponding to a Trace object, a continuous part of the wave).

In [12]:
def gaussian_interpolation(v1, v2, nb_missing_values, mean=0, std=300, sample_freq=SPS):
    x = [0, nb_missing_values]
    f_x = [v1, v2]
    missing_times = np.linspace(0, nb_missing_values - 1, nb_missing_values)
    interp_v = np.interp(missing_times, x, f_x)
    noise = np.random.normal(mean, std, nb_missing_values)
    return interp_v + noise

def load_raw_X(data_path, dirs='all'):
    """
    This function load data as it is download by the scripts download_datasets.py, that is every Trace
    object of obspy written to disk as SLIST format in subdirectories of the data_path directory. This
    directory should only contain subdirectories of data. Those subdirectories corresponds to 15 days 
    of data and the chunks that is in them must be consecutive, the directory data/2/ must contain the
    15 days that follows the content of data/1/.
    
    data_path: the directory containing all subdirectories of data
    dirs: the subdirectories to take into account, maximum possible is ~6 with 20GB of RAM
    
    return: (X, start_time)
            X: the raw data as a one-dimensionnal numpy array
            start_time: the time at which this chunk of data starts
    """
    if dirs == 'all':
        dirs = !ls -v $data_path 
    
    X = np.array([])
    for i_dir, d in enumerate(dirs):
        print(f"Loading days {15*i_dir+1} to {15*(i_dir+1) - 1}")
        dir_path = os.path.join(data_path+'/', d)
        if i_dir == 0:
            first_trace_header = pd.read_csv(dir_path+'/'+'0.slist', sep='\t', nrows=0).columns[0].split(', ')
            train_start_time = pd.to_datetime(first_trace_header[START_TIME_INDEX])
        
        first_trace_header = pd.read_csv(dir_path+'/'+'0.slist', sep='\t', nrows=0).columns[0].split(', ')
        dir_start_time = pd.to_datetime(first_trace_header[START_TIME_INDEX])

        dir_X = np.array([])
        all_slists = !ls -v $dir_path
        for slist in all_slists:
            
            # Compute size of padding
            slist_full_path = os.path.join(dir_path, slist)
            header = pd.read_csv(slist_full_path, sep='\t', nrows=0).columns[0].split(', ')
            time_elapsed =  pd.to_datetime(header[START_TIME_INDEX]) - dir_start_time
            num_elem_elapsed = int(time_elapsed.total_seconds() * SPS)
            num_elem_padd = num_elem_elapsed - dir_X.shape[0]

            # Extract data in current file
            new_X = pd.read_csv(slist_full_path, sep='\t', header=0, names=["1", "2", "3", "4", "5", "6"])
            new_X = new_X.to_numpy().reshape(-1)
            new_X = new_X[~(np.isnan(new_X))]

            # Add interpolated padding and new data
            prev = dir_X[-1] if dir_X.shape[0] != 0 else 0
            value_interpol = (prev + new_X[0]) / 2
            gaussian_padding = gaussian_interpolation(value_interpol, value_interpol, num_elem_padd)
            dir_X = np.concatenate((dir_X,gaussian_padding))
            dir_X = np.concatenate((dir_X, new_X))
        X = np.concatenate((X, dir_X))
        print(X.shape[0])
        mean_prev = np.mean(X[-100:])
        num_elem_padd_dir = np.max([(i_dir+1)*NUM_DAY_PER_DIR*24*3600*100 - X.shape[0], 0])
        X = np.concatenate((X, gaussian_interpolation(mean_prev, mean_prev, num_elem_padd_dir)))
            
    return X, train_start_time

Now that we know how to load the data downloaded using the scripts, we create the functions that will compute the features

In [13]:
def basic_features(X):
    """
    This computes some basic features:
    - Standard deviation of the signal
    - Maximum and minimum of the signal
    - Maximum difference between two adjacent points
    - Maximum and minimum of the shifted signal
    - Number of points that are above half of the max (~number of points with high values)
    - Number of points that are below half of the min (~number of points with low values)
    """
    N = X.shape[0]
    
    f_1 = np.std(X, axis=1).reshape((N, 1))
    f_2 = np.max(X, axis=1).reshape((N, 1))
    f_3 = np.min(X, axis=1).reshape((N, 1))
    X_mean = X - np.mean(X, axis=1).reshape((N, 1))
    f_4 = np.max(X_mean[:,:-1] - X_mean[:,1:], axis=1).reshape((N, 1))
    f_5 = np.max(X_mean, axis=1).reshape((N, 1))
    f_6 = np.min(X_mean, axis=1).reshape((N, 1))
    f_7 = np.sum(X_mean > f_5/2, axis=1).reshape((N, 1))
    f_8 = np.sum(X_mean < f_6/2, axis=1).reshape((N, 1))
    
    return np.hstack((f_1, f_2, f_3, f_4, f_5, f_6, f_7, f_8))

In [14]:
def count_abrupt_changes(X, amplitude):
    """
    This computes the number of times two consecutive points differ by a great amount.
    This is done by counting the number of times that two consecutive points change
    of sign, by a sufficient amount.
    """
    X = X - np.mean(X, axis=1).reshape(X.shape[0], 1)
    first_kernel = X[:,1:]
    second_kernel = X[:,:-1]
    change_of_sign = first_kernel * second_kernel
    
    return np.sum(change_of_sign < -amplitude, axis=1).reshape(X.shape[0], 1)

In [15]:
def weighted_sliding_mean(X, mean_size):
    """
    This computes the maximum difference between two consecutive points, weighted by the 
    mean of the previous changes.
    """
    N, W = X.shape
    diff = np.abs(X[:, 1:] - X[:, :-1])
    diff = diff[:, mean_size-1:]
    
    diff_sliding_mean = np.zeros((N, W - (mean_size - 1)))
    for i, w in enumerate(X):
        diff_sliding_mean[i] = np.abs(np.convolve(w, np.ones(mean_size), 'valid') / mean_size)
    weighted_diff = diff / (diff_sliding_mean[:,:-1] + 1)
    
    return np.max(weighted_diff, axis=1).reshape((N, 1))

In [16]:
def fourier_features(X, min_freq):
    """
    This will compute 5 features from the fourier transform:
    - The median of the amplitude in the spectrum
    - The mean of the amplitude in the spectrum
    - The standard deviation of the amplitude, taking spectrum from 20Hz
    - The standard deviation of the amplitude, taking spectrum from 2.5Hz to 20Hz
    - The mean of the amplitude, taking spectrum from 2.5Hz to 20Hz
    """
    # Number of samplepoints
    N = X.shape[0]

    # sample spacing
    T = 1.0 / 200 # Want frequency until 100hz
    
    X = X - np.mean(X, axis=1).reshape((N, 1))
    xf = np.fft.fft(X, axis=1)
    x_amplitudes = 2.0/N * np.abs(xf[:, :N//2])
    medians = np.median(x_amplitudes, axis=1).reshape(N, 1)
    means = np.mean(x_amplitudes[:, min_freq:], axis=1).reshape(N, 1)
    
    split_1 = int(x_amplitudes.shape[1]/5) # 20hz
    split_2 = int(x_amplitudes.shape[1]/40) # 2.5hz
    
    f_1 = np.std(x_amplitudes[:,split_1:],axis=1).reshape(N,1)
    f_2 = np.std(x_amplitudes[:,split_2:split_1],axis=1).reshape(N,1)
    f_3 = np.mean(x_amplitudes[:,split_2:split_1],axis=1).reshape(N,1)
    
    return np.hstack((medians, means, f_1, f_2, f_3))

We define now the function that aggregate all functions of this sections and the catalog to create the X and y in an adequate form for an algorithm.

In [17]:
def compute_end_time(number_sample, start_time):
    total_number_of_seconds = (number_sample - 1) / SPS
    end_time = start_time + pd.to_timedelta(total_number_of_seconds, unit='s')

    return end_time

def compute_X_and_y(filename, SPS, catalog, amplitude=100000, dirs='all', preloaded_data_paths=None):
    window_size = TIME_WINDOW*SPS
    if preloaded_data_paths == None:
        X, start_time = load_raw_X(filename, dirs=dirs)

        N = X.shape[0]
        print(f"Number of points {N}")
        number_window = np.math.floor(N / window_size)
        X = X[:number_window*window_size]
        X_time_window = X.reshape(number_window, window_size)
        print(f"X shape {X_time_window.shape}")
    else:
        X_raw_path, metadata_path = preloaded_data_paths
        metadata = pd.read_csv(metadata_path)
        start_time = pd.to_datetime(metadata.loc[0, '0'])
        
        X_time_window = pd.read_hdf(X_raw_path).to_numpy()
        print(f"X shape {X_time_window.shape}")
        
    end_time  = compute_end_time(X_time_window.shape[0]*X_time_window.shape[1], start_time)
    catalog = catalog[(catalog["origintime"] >= start_time) & (catalog["origintime"] <= end_time)]
    
    y = np.zeros((X_time_window.shape[0], 1))
    
    for date in catalog["origintime"]:
        seconds_to_eq = (date - start_time).total_seconds()
        index_in_data = seconds_to_eq * SPS
        index = int(index_in_data / window_size)
        y[index] = catalog.loc[catalog['origintime'] == date, 'magnitude']
    
    feature1 = count_signs_per_row(X_time_window, amplitude)
    feature2 = weighted_sliding_mean_per_row(X_time_window, 25)
    feature3_7 = fourier_features(X_time_window, 30)
    feature8_15 = basic_features(X_time_window)
    
    
    return X_time_window, np.hstack((feature1, feature2, feature3_7, feature8_15)), y, start_time, end_time

def write_X_and_y(X, y, start, end, start_number, end_number):
    pd.DataFrame(X).to_csv(f'processed-data/X_{start_number}_to_{end_number}.csv')
    pd.DataFrame(y).to_csv(f'processed-data/y_{start_number}_to_{end_number}.csv')
    pd.DataFrame([start, end]).to_csv(f'processed-data/meta_{start_number}_to_{end_number}.csv')
    
def load_X_and_y(minimum_magnitude=0):
    X = pd.DataFrame()
    y = pd.DataFrame()
    for i in [1, 5, 9]:
        X = X.append(pd.read_csv(f'processed-data/X_{i}_to_{i+3}.csv', index_col=0))
        y = y.append(pd.read_csv(f'processed-data/y_{i}_to_{i+3}.csv', index_col=0))
    X, y = X.to_numpy(), y.to_numpy()
        
    mask_earthquake = y > minimum_magnitude
    y[mask_earthquake] = 1
    y[~mask_earthquake] = 0
    return X, y.ravel()

## 3. Creation of the model

In [48]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

def show_accuracy(pred, true, set_):
    acc = accuracy_score(true, pred)
    prec = precision_score(true, pred)
    rec = recall_score(true, pred)
    f1 = f1_score(true, pred)
    print(f"Predicted earthquakes: {pred[pred==1].shape[0]} of {true.sum()}")
    print(f"{set_} Accuracy: {acc}")
    print(f"{set_} Precision: {prec}")
    print(f"{set_} Recall: {rec}")
    print(f"{set_} F1 score: {f1}")
    return [acc, prec, rec, f1]
    
def predict(model):
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    show_accuracy(pred_train, y_train, "Train")
    print("")
    return show_accuracy(pred_test, y_test, "Test")
    
X, y = load_X_and_y(minimum_magnitude=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Total number of aggregated sample: {X.shape[0]}")

Total number of aggregated sample: 237600


Training with all earthquakes and random forest

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression

print("proportion of earthquake:")
print("\ttrain: ", y_train.sum(), "out of", y_train.shape[0], f"({y_train.sum()/y_train.shape[0]*100:.2f}%)")
print("\ttest", y_test.sum(), "out of", y_test.shape[0], f"({y_test.sum()/y_test.shape[0]*100:.2f}%)")

print("RF:")
for (d, w, c) in [[43, {0: 1, 1: 10000}, 100], [100,{0: 1, 1: 10000}, 100]]:
    print("\n ----- With parameters: ", d, w, c, ' ----- ')
    rfc = RandomForestClassifier(n_estimators=c, max_depth=d, class_weight=w, n_jobs=4)
    rfc.fit(X_train, y_train)
    predict(rfc)

proportion of earthquake:
	train:  857.0 out of 190080 (0.45%)
	test 218.0 out of 47520 (0.46%)
RF:

 ----- With parameters:  43 {0: 1, 1: 10000} 100  ----- 
Predicted earthquakes: 4660 of 857.0
Train Accuracy: 0.9799821127946128
Train Precision: 0.18369098712446352
Train Recall: 0.9988331388564761
Train F1 score: 0.31031357621895955

Predicted earthquakes: 1002 of 218.0
Test Accuracy: 0.9770622895622896
Test Precision: 0.06487025948103792
Test Recall: 0.2981651376146789
Test F1 score: 0.10655737704918032

 ----- With parameters:  100 {0: 1, 1: 10000} 100  ----- 
Predicted earthquakes: 854 of 857.0
Train Accuracy: 0.9999842171717171
Train Precision: 1.0
Train Recall: 0.9964994165694282
Train F1 score: 0.9982466393921683

Predicted earthquakes: 10 of 218.0
Test Accuracy: 0.9953703703703703
Test Precision: 0.4
Test Recall: 0.01834862385321101
Test F1 score: 0.03508771929824561


Training with all earthquakes with magnitude of at least 2 and random forest

In [ ]:
X, y = load_X_and_y(minimum_magnitude=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Total number of aggregated sample: {X.shape[0]}")

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression

print("proportion of earthquake:")
print("\ttrain: ", y_train.sum(), "out of", y_train.shape[0], f"({y_train.sum()/y_train.shape[0]*100:.2f}%)")
print("\ttest", y_test.sum(), "out of", y_test.shape[0], f"({y_test.sum()/y_test.shape[0]*100:.2f}%)")

print("RF:")
for (d, w, c) in [[43, {0: 1, 1: 10000}, 100]]:
    print("\n ----- With parameters: ", d, w, c, ' ----- ')
    rfc = RandomForestClassifier(n_estimators=c, max_depth=d, class_weight=w, n_jobs=4)
    rfc.fit(X_train, y_train)
    predict(rfc)

proportion of earthquake:
	train:  614.0 out of 190080 (0.32%)
	test 138.0 out of 47520 (0.29%)
RF:

 ----- With parameters:  43 {0: 1, 1: 10000} 100  ----- 
Predicted earthquakes: 958 of 614.0
Train Accuracy: 0.9981902356902357
Train Precision: 0.6409185803757829
Train Recall: 1.0
Train F1 score: 0.7811704834605598

Predicted earthquakes: 128 of 138.0
Test Accuracy: 0.9956649831649832
Test Precision: 0.234375
Test Recall: 0.21739130434782608
Test F1 score: 0.2255639097744361


Training with all earthquakes of magnitude at least 3 and random forest

In [81]:
X, y = load_X_and_y(minimum_magnitude=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(f"Total number of aggregated sample: {X.shape[0]}")

Total number of aggregated sample: 237600


In [65]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression

print("proportion of earthquake:")
print("\ttrain: ", y_train.sum(), "out of", y_train.shape[0], f"({y_train.sum()/y_train.shape[0]*100:.2f}%)")
print("\ttest", y_test.sum(), "out of", y_test.shape[0], f"({y_test.sum()/y_test.shape[0]*100:.2f}%)")

print("RF:")
for (d, w, c) in [[45, {0: 1, 1: 15000}, 100]]:
    print("\n ----- With parameters: ", d, w, c, ' ----- ')
    rfc = RandomForestClassifier(n_estimators=c, max_depth=d, class_weight=w, n_jobs=4)
    rfc.fit(X_train, y_train)
    predict(rfc)

proportion of earthquake:
	train:  56.0 out of 190080 (0.03%)
	test 7.0 out of 47520 (0.01%)
RF:

 ----- With parameters:  45 {0: 1, 1: 15000} 100  ----- 
Predicted earthquakes: 56 of 56.0
Train Accuracy: 1.0
Train Precision: 1.0
Train Recall: 1.0
Train F1 score: 1.0

Predicted earthquakes: 2 of 7.0
Test Accuracy: 0.9998947811447811
Test Precision: 1.0
Test Recall: 0.2857142857142857
Test F1 score: 0.4444444444444445


In [83]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression

print("proportion of earthquake:")
print("\ttrain: ", y_train.sum(), "out of", y_train.shape[0], f"({y_train.sum()/y_train.shape[0]*100:.2f}%)")
print("\ttest", y_test.sum(), "out of", y_test.shape[0], f"({y_test.sum()/y_test.shape[0]*100:.2f}%)")

print("RF:")
for (d, w, c) in [[45, {0: 1, 1: 15000}, 100]]:
    print("\n ----- With parameters: ", d, w, c, ' ----- ')
    rfc = RandomForestClassifier(n_estimators=c, max_depth=d, class_weight=w, n_jobs=4)
    rfc.fit(X_train, y_train)
    predict(rfc)

proportion of earthquake:
	train:  57.0 out of 213840 (0.03%)
	test 6.0 out of 23760 (0.03%)
RF:

 ----- With parameters:  45 {0: 1, 1: 15000} 100  ----- 
Predicted earthquakes: 57 of 57.0
Train Accuracy: 1.0
Train Precision: 1.0
Train Recall: 1.0
Train F1 score: 1.0

Predicted earthquakes: 2 of 6.0
Test Accuracy: 0.9998316498316498
Test Precision: 1.0
Test Recall: 0.3333333333333333
Test F1 score: 0.5


In [42]:
from sklearn.neighbors import KNeighborsClassifier

scores = []

for k in range(1, 11):
    for n in range(11):
        X_train_copy = X_train.copy()
        y_train_copy = y_train.copy()
        
        for i in range(n):
            X_train_copy = np.vstack((X_train_copy, X_train[y_train==1]))
            y_train_copy = np.hstack((y_train_copy, y_train[y_train==1]))
            
        print(f"Number of copies: {n}")
        print(f"Number of neighbors: {k}")
        print(X_train_copy.shape)
        print((y_train_copy == 1).sum())
        
        knn = KNeighborsClassifier(n_neighbors=k, n_jobs=8)
        knn.fit(X_train_copy, y_train_copy)
        scores.append(predict(knn))

Number of copies: 0
Number of neighbors: 1
(190080, 15)
841
Predicted earthquakes: 841, from which 841 were correct from the true 841.0 earthquakes
Train Accuracy: 1.0
Train Precision: 1.0
Train Recall: 1.0
Train F1 score: 1.0

Predicted earthquakes: 183, from which 18 were correct from the true 234.0 earthquakes
Test Accuracy: 0.9919823232323233
Test Precision: 0.09836065573770492
Test Recall: 0.07692307692307693
Test F1 score: 0.08633093525179857
Number of copies: 1
Number of neighbors: 1
(190921, 15)
1682
Predicted earthquakes: 841, from which 841 were correct from the true 841.0 earthquakes
Train Accuracy: 1.0
Train Precision: 1.0
Train Recall: 1.0
Train F1 score: 1.0

Predicted earthquakes: 183, from which 18 were correct from the true 234.0 earthquakes
Test Accuracy: 0.9919823232323233
Test Precision: 0.09836065573770492
Test Recall: 0.07692307692307693
Test F1 score: 0.08633093525179857
Number of copies: 2
Number of neighbors: 1
(191762, 15)
2523
Predicted earthquakes: 841, from

In [44]:
scores

[0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.024390243902439022,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.086330935251798566,
 0.068702290076335867,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.040816326530612249,
 0.068702290076335867,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.080906148867313912,
 0.08090614

In [45]:
np.max(scores)

0.086330935251798566